# Machine Learning Assignment - 1

## Part - II: Programming

### by

<b>Mohammed Ilyas Habeeb</b> and <b>Faiyaz Sundrani</b>

In [1]:
import math
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

# Loading the Data

In [4]:
#Reading the data
data = pd.read_csv('glasshw1.csv', header = None, names = ['indextoUse', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8',
                                                           'x9', 'Class'], index_col = 'indextoUse')

# data = pd.read_csv('glasshw1.csv', header = None, names = ['indextoUse', 'RI', 'Na', 'Mg', 'Al', 'Si', 'K', 'Ca', 'Ba',
#                                                            'Fe', 'Class'], index_col = 'indextoUse')

In [5]:
#Viewing the head of data
data.head()

,x1,x2,x3,x4,x5,x6,x7,x8,x9,Class
indextoUse,,,,,,,,,,
1,1.51824,12.87,3.48,1.29,72.95,0.60,8.43,0.00,0.00,1
2,1.51832,13.33,3.34,1.54,72.14,0.56,8.99,0.00,0.00,1
3,1.51747,12.84,3.50,1.14,73.27,0.56,8.55,0.00,0.00,1
4,1.51775,12.85,3.48,1.23,72.97,0.61,8.56,0.09,0.22,1
5,1.51768,12.65,3.56,1.30,73.08,0.61,8.69,0.00,0.14,1


# Training the Data

We train the data by estimating the mean and variance of each class

In [6]:
#Calculating the prior
def PriorCalculation(data_to_use):
    '''
    i/p : the class column, list of ints (either 1 or 2)
    o/p: prior of the given class, a float
    '''
    res_Class1 = 0
    res_Class2 = 0
    for i in data_to_use:
        if i == 1:
            res_Class1 += 1
        elif i == 2:
            res_Class2 += 1
    prior_res_Class1 = res_Class1 / len(data_to_use)
    prior_res_Class2 = res_Class2 / len(data_to_use)
    
    return prior_res_Class1, prior_res_Class2
    

In [7]:
prior_Class1, prior_Class2 = PriorCalculation(data['Class'])

#printing the priors
print("Prior P(C = 1):", prior_Class1)
print("Prior P(C = 2):", prior_Class2)

Prior P(C = 1): 0.745
Prior P(C = 2): 0.255


In [8]:
#Calculating the mean and variance
def meanAndVarianceCalculation(col_attribute, col_class, class_value):
    '''
    i/p:
    column attribute - the attribute for which mean and variance is computed
    column class - the class column for which mean and variance is computed
    class value - the value of the class (either 1 or 2)
    o/p:
    mean, variance - float
    '''
    result_mean = 0
    result_variance = 0
    len_class = 0
    for i, j in zip(col_attribute, col_class):
        if j == class_value:
            result_mean = result_mean + i
            len_class += 1
        
    mean = result_mean / len_class
    
    for i, j in zip(col_attribute, col_class):
        if j == class_value:
            result_variance = result_variance + (i - mean) ** 2
            
    
    variance = result_variance / ( (len_class) - 1)
    
    return mean, variance

In [9]:
#Function to pass values to mean and variance to meanAndVarianceCalculation function
def meanVariancePerClass(data):
    '''
    i/p: a dataset 'data'
    o/p: lists of mean and variance of both the classes
    '''
    mClass1 = []
    mClass2 = []
    vClass1 = []
    vClass2 = []
    #Passing attributes (x1, x2, ....,x9) to meanAndVarianceCalculation function
    for i in range(1, 10):
        x = "x"+str(i)
        
        mc1, vc1 = meanAndVarianceCalculation(data[x], data['Class'], 1)
        #Appending results to lists
        mClass1.append(mc1)
        vClass1.append(vc1)
        
        mc2, vc2 = meanAndVarianceCalculation(data[x], data['Class'], 2)
        mClass2.append(mc2)
        vClass2.append(vc2)
        
    return mClass1, mClass2, vClass1, vClass2 
        
        

In [10]:
mean_Class1, mean_Class2, variance_Class1, variance_Class2 = meanVariancePerClass(data)

In [11]:
#Renaming the indices
index_meanvariance = ['RI', 'Na', 'Mg', 'Al', 'Si', 'K', 'Ca', 'Ba', 'Fe']
#Making a DataFrame of mean and variances
meanvariance_data = DataFrame(
            {'Mean_Class1': mean_Class1,
             'Variance_Class1': variance_Class1,
             'Mean_Class2': mean_Class2,
             'Variance_Class2': variance_Class2
            }, index = index_meanvariance)

meanvariance_data = meanvariance_data[['Mean_Class1', 'Variance_Class1', 'Mean_Class2', 'Variance_Class2']]

meanvariance_data

,Mean_Class1,Variance_Class1,Mean_Class2,Variance_Class2
RI,1.518662,0.000010,1.517638,0.000008
Na,13.198859,0.364710,14.066667,1.138139
Mg,3.271812,0.855103,0.733725,1.213176
Al,1.276309,0.109656,1.966667,0.357759
Si,72.578456,0.427340,72.855490,1.167665
K,0.470403,0.050639,0.559608,1.651756
Ca,8.968725,2.013964,9.060196,2.509274
Ba,0.031879,0.070230,0.639216,0.556411
Fe,0.070872,0.010430,0.023137,0.006826


# Testing the Data

We test the data by predicting the class by calculating the MAP for each observation

In [12]:
#Function to calculate likelihood
def likelihoodCalculator(x, mean, variance):
    '''
    i/p: 
    x - value of attribute of test dataset
    mean - mean of attribute of the corresponding training dataset
    variance - variance of attribute of the corresponding dataset
    o/p:
    ans - likelihood, a float
    '''
    res = ( x - mean) ** 2
    
    exp_power = (-1/(2*variance)) * res
    taking_exp = np.exp(exp_power)
    
    ans = ( 1/(math.sqrt(2*np.pi*variance)) ) * taking_exp
    
    return ans
    

In [11]:
#dropping the target variable, i.e., the class column
data_only_attributes = data.drop('Class', axis = 1)
#len(data_only_attributes.loc[1])

In [12]:
#A function that passes the value, mean, and variance to the likelihoodCalculator function
def logLikelihoodCalculator(values, mean_x, variance_x, prior):
    '''
    i/p:
    values - a list of attributes [x1, x2, ....., x9]
    mean_x - a list of means for the attributes in values
    variance_x - a list of variances for the attributes in values
    prior - the prior value of a given class
    o/p:
    final_ans - the log posterior
    '''
    #list to store the log of likelihoods
    log_p_xi = []

    for i in range(0, len(values)):
        x = values[i]#the 
        m_x = mean_x[i]
        v_x = variance_x[i]
        #passing the individual value, mean, variance one at a time to the likelihood Calculator
        p_x = likelihoodCalculator(x, m_x, v_x)
        #testing for condition if the likelihood is zero
        if p_x != 0:
            #if the likelihood is not zero, then only append it to the list and apply log to the likelihood
            log_p_xi.append(np.log(p_x))
        else:
            log_p_xi.append(np.log(p_x + 1000))
            
    #taking the sum of the log likelihoods in the list and storing it in sum_log
    sum_log = np.sum(log_p_xi)
    
    #applying log to the prior and adding it to sum_log
    final_ans = np.log(prior) + sum_log
    
    #returning the final answer, which is the log posterior
    return final_ans


In [13]:
#A function that predicts classes
def predictClasses(data_attributes, mean1, variance1, prior1, mean2, variance2, prior2):
    '''
    i/p:
    data_attributes - the dataset which contains only the attributes, i.e., (x1, x2, ...., x9)
    mean1 - a list which contains the means of the attributes in the training dataset for class 1
    variance1 - a list which contains the variances of the attributes in the training dataset for class 1
    mean2 - a list which contains the means of the attributes in the training dataset for class 2
    variance2 - a list which contains the variances of the attributes in the training dataset for class 2
    o/p:
    logPosterior_Class1 - a list which contains the calculated the logPosteriors of class 1
    logPosterior_Class2 - a list which contains the calculated the logPosteriors of class 2
    predictions - a list of prediction of classes,i.e., predicting the classes of test dataset as either 1 or 2
    '''
    logPosterior_Class1 = []
    logPosterior_Class2 = []
    predictions = []
    
    #ranging row wise
    for i in range(data_attributes.index[0], data_attributes.index[len(data_attributes.index)-1] + 1):
        
        #per row, pass the [x1, x2, ....., x9] values to logLikelihoodCalculator for calculating log Posteriors for Class 1
        res_Class1 = logLikelihoodCalculator(data_attributes.loc[i], mean1, variance1, prior1)
        logPosterior_Class1.append(res_Class1)
        
        #per row, pass the [x1, x2, ....., x9] values to logLikelihoodCalculator for calculating log Posteriors for Class 2
        res_Class2 = logLikelihoodCalculator(data_attributes.loc[i], mean2, variance2, prior2)
        logPosterior_Class2.append(res_Class2)
    
    #iterating over the lists of log Posteriors to compare values
    for i, j in zip(logPosterior_Class1, logPosterior_Class2):
        #if value in logPosterior_Class1 is greater, append 1 as prediction
        if i >= j:
            predictions.append(1)
        #else, append 2 as prediction
        else:
            predictions.append(2)
    
    #return the lists of logPosteriors for both classes and predictions
    return logPosterior_Class1, logPosterior_Class2, predictions



logPosterior_Class1, logPosterior_Class2, predictions = predictClasses(data_only_attributes, mean_Class1, variance_Class1, 
                                                                  prior_Class1, mean_Class2, variance_Class2, prior_Class2)


In [14]:
#printing the list of predictions
print("Predictions made on each of the 200 examples:")
print(predictions)

Predictions made on each of the 200 examples:
[1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 2, 2, 1, 2, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1, 2, 2, 1, 1, 2, 1, 1, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 2, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 2, 2, 1, 1, 2, 1, 1, 1, 1, 1]


In [15]:
#Taking a copy of the original dataset
data_output = data.copy()

#Adding the predictions list as a column for the 'data_output' dataset
data_output['Predicted_Class_GNB'] = predictions

#viewing the data_output
data_output.head()

,x1,x2,x3,x4,x5,x6,x7,x8,x9,Class,Predicted_Class_GNB
indextoUse,,,,,,,,,,,
1,1.51824,12.87,3.48,1.29,72.95,0.60,8.43,0.00,0.00,1,1
2,1.51832,13.33,3.34,1.54,72.14,0.56,8.99,0.00,0.00,1,1
3,1.51747,12.84,3.50,1.14,73.27,0.56,8.55,0.00,0.00,1,1
4,1.51775,12.85,3.48,1.23,72.97,0.61,8.56,0.09,0.22,1,1
5,1.51768,12.65,3.56,1.30,73.08,0.61,8.69,0.00,0.14,1,1


In [16]:
#A function to calculate training error and accuracy
def trainingErrorAndAccuracy(actual_Class, predicted_Class):
    '''
    i/p:
    actual_Class - passing the list of actual classes
    predicted_Class - passing the list of predicted classes
    o/p:
    error_percentage - the training error
    accuracy_percentage - the accuracy
    '''
    
    misclassified = 0
    classified = 0
    #iterating over both the classes
    for i, j in zip(actual_Class, predicted_Class):
        #if both the values don't match, treat as misclassified
        if i != j:
            misclassified += 1
        #else, if they are same, treat as classified
        else:
            classified += 1
    
    #calculating error as [number of incorrect predictions]/[total number of class values]
    error = misclassified / len(actual_Class)
    
    #calculating error as [number of correct predictions]/[total number of class values]
    accuracy = classified / len(actual_Class)
    
    #multiply error by 100 to get error percentage
    error_percentage = error * 100
    
    #multiply error by 100 to get accuracy percentage
    accuracy_percentage = accuracy * 100
    
    return error_percentage, accuracy_percentage

In [17]:
training_error_entire_set, accuracy_entire_set = trainingErrorAndAccuracy(list(data['Class']), predictions)

#Printing the training error and accuracy percentages
print("The Training Error % on 200 examples is:", training_error_entire_set)
print("The Accuracy % on 200 examples is:", accuracy_entire_set)

The Training Error % on 200 examples is: 10.0
The Accuracy % on 200 examples is: 90.0


# 5-Fold Cross Validation

Before, we were training and testing using the entire 200 examples. <br /> <br />
Now, we will train one set of examples and test it on the other sets. <br /> <br />
Since we are using 5-fold cross validation, we will: <br /> <br />
    i) Test the 1st fold, train on the 2nd, 3rd, 4th, and 5th folds <br />
    ii) Test the 2nd fold, train on the 1st, 3rd, 4th, and 5th folds <br />
    iii) Test the 3rd fold, train on the 1st, 2nd, 4th, and 5th folds <br />
    iv) Test the 4th fold, train on the 1st, 2nd, 3rd, and 5th folds <br />
    v) Test the 5th fold, train on the 1st, 2nd, 3rd, and 4th folds

In [18]:
#A function that does cross validation and returns error, accuracy, and predictions
def crossValidationwithGNB(data, k):
    '''
    i/p:
    data - the dataset 'data'
    k - number of folds
    o/p:
    avg_error - average training error of entire folds combined
    avg_accuracy - average accuracy of entire folds combined
    predictions_list - a list of predicted values
    '''
    #getting the number of samples to take per fold by dividing total observations by k
    samples = int(len(data)/k)
    
    #lists to store error, accuracy, and predictions
    error_list = []
    accuracy_list = []
    predictions_list = []
    
    #iterating fold-wise
    for i in range(1, k+1):
        
        #taking one fold as test_set
        test_set = data[(i-1)*samples:i*samples]
        #taking the rest of the folds as the training_set
        training_set = pd.concat([data[:(i-1)*samples], data[i*samples:]])
        
        #calculating the prior of the classes by passing Class column in training_set to PriorCalculation function
        prior_Class1_training, prior_Class2_training = PriorCalculation(training_set['Class'])
        
#         print("Prior for Class 1 at Fold", i, "is:", prior_Class1_training)
#         print("Prior for Class 1 at Fold", i, "is:", prior_Class2_training)
        
        #calculating means and variances of both the classes of the training_set by passing values to
        #meanVariancePerClass function
        mean_Class1, mean_Class2, variance_Class1, variance_Class2 = meanVariancePerClass(training_set)
        
        #dropping the target variable column, i.e., the class column
        test_only_attributes = test_set.drop('Class', axis = 1)
        
        #calculating the logPosteriors of both the classes and predictions by passing 
        #test values, means and variances of training_set, and priors of both classes of training_set 
        #to predictClasses function
        logPosterior_Class1, logPosterior_Class2, predictions = predictClasses(test_only_attributes, 
                                                                mean_Class1, variance_Class1, 
                                                        prior_Class1_training, mean_Class2, variance_Class2, 
                                                                       prior_Class2_training)
        #put the predictions result in the predictions list
        predictions_list.extend(predictions)
        
        #calculate the training error and accuracy by passing actual_classes and predicted_classes 
        #to trainingErrorAndAccuracy classes
        training_errorset, accuracy_set = trainingErrorAndAccuracy(list(test_set['Class']), predictions)
        
        #Printing the training error and accuracy of each fold
        print("At Fold", i, ":", "Training Error is", training_errorset)
        print("At Fold", i, ":", "Accuracy is", accuracy_set)

        #appending training error and accuracy of each fold to lists
        error_list.append(training_errorset)
        accuracy_list.append(accuracy_set)
        
    #calculating average training error and average accuracy    
    avg_error = sum(error_list)/len(error_list)
    avg_accuracy = sum(accuracy_list)/len(accuracy_list)
    
    
    return avg_error, avg_accuracy, predictions_list
        
    
    

In [19]:
final_crossvalidation_error, final_crossvalidation_accuracy, predictions_CrossValidation = crossValidationwithGNB(data, 5)

#making a column of predicted values
data_output['Predicted_Class_CV_GNB'] = predictions_CrossValidation

#Printing predictions
print("Predictions with cross validation for GNB is:") 
print(predictions_CrossValidation)
print("Final Training Error % with cross validation is", final_crossvalidation_error)
print("Final Accuracy % with cross validation is", final_crossvalidation_accuracy)

At Fold 1 : Training Error is 7.5
At Fold 1 : Accuracy is 92.5
At Fold 2 : Training Error is 12.5
At Fold 2 : Accuracy is 87.5
At Fold 3 : Training Error is 5.0
At Fold 3 : Accuracy is 95.0
At Fold 4 : Training Error is 15.0
At Fold 4 : Accuracy is 85.0
At Fold 5 : Training Error is 10.0
At Fold 5 : Accuracy is 90.0
Predictions with cross validation for GNB is:
[1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 2, 2, 1, 2, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1, 2, 2, 1, 1, 2, 1, 1, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 2, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 2, 2, 1, 1, 2, 1, 1, 1, 1, 1]
Final Training Error % with cross v

In [20]:
#A function to calculate cross Validation with Zero-R algorithm
def crossValidationwithZeroR(data, k):
    '''
    i/p:
    data - the dataset 'data'
    k - number of folds
    o/p:
    avg_error: average training error of the folds
    avg_accuracy: average accuracy of the folds
    predictions_list: list of predictions
    '''
    samples = int(len(data)/k)
    error_list = []
    accuracy_list = []
    predictions_list = []
    for i in range(1, k+1):
        
        test_set = data[(i-1)*samples:i*samples]
        training_set = pd.concat([data[:(i-1)*samples], data[i*samples:]])
        
        count_C1 = 0
        count_C2 = 0
        
        #calculating the counts of Classes 1 and Classes 2
        for j in training_set['Class']:
            if j == 1:
                count_C1 += 1
            elif j == 2:
                count_C2 += 1
        #if count_C1 > count_C2:
        #if count_C1 > count_C2
        #predicting the Class in test_set which has the maximum count in the training_set
        predictions = [1 if count_C1 > count_C2 else 2 for j in test_set['Class'] ]
        
        #calculating the training error and accuracy of each fold
        training_errorset, accuracy_set = trainingErrorAndAccuracy(list(test_set['Class']), predictions)
        
        print("At Fold", i, ":", "Training Error is", training_errorset)
        print("At Fold", i, ":", "Accuracy is", accuracy_set)
        
        #appending predictions and training error and accuracy
        predictions_list.extend(predictions)
        error_list.append(training_errorset)
        accuracy_list.append(accuracy_set)
        
    #taking the average training error and accuracy    
    avg_error = sum(error_list)/len(error_list)
    avg_accuracy = sum(accuracy_list)/len(accuracy_list)
    
    return avg_error, avg_accuracy, predictions_list

In [21]:
zero_R_error, zero_R_accuracy, predictions_list_CVZeroR = crossValidationwithZeroR(data, 5)

#making a column of predicted values
data_output['Predicted_Class_CV_ZeroR'] = predictions_list_CVZeroR

#printing predictios
print("Predictions with cross validation for Zero-R is:") 
print(predictions_list_CVZeroR)
print("Final Training Error % with cross validation using Zero-R is:", zero_R_error)
print("Final Accuracy % with cross validation using Zero-R is:", zero_R_accuracy)

At Fold 1 : Training Error is 27.500000000000004
At Fold 1 : Accuracy is 72.5
At Fold 2 : Training Error is 17.5
At Fold 2 : Accuracy is 82.5
At Fold 3 : Training Error is 25.0
At Fold 3 : Accuracy is 75.0
At Fold 4 : Training Error is 27.500000000000004
At Fold 4 : Accuracy is 72.5
At Fold 5 : Training Error is 30.0
At Fold 5 : Accuracy is 70.0
Predictions with cross validation for Zero-R is:
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Fi

In [22]:
data_output.columns = ['RI', 'Na', 'Mg', 'Al', 'Si', 'K', 'Ca', 'Ba', 'Fe', 'Class', 'Predicted_Class_GNB',
                      'Predicted_Class_CV_GNB', 'Predicted_Class_CV_ZeroR']

data_output.head(20)

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Class,Predicted_Class_GNB,Predicted_Class_CV_GNB,Predicted_Class_CV_ZeroR
indextoUse,,,,,,,,,,,,,
1,1.51824,12.87,3.48,1.29,72.95,0.60,8.43,0.00,0.00,1,1,1,1
2,1.51832,13.33,3.34,1.54,72.14,0.56,8.99,0.00,0.00,1,1,1,1
3,1.51747,12.84,3.50,1.14,73.27,0.56,8.55,0.00,0.00,1,1,1,1
4,1.51775,12.85,3.48,1.23,72.97,0.61,8.56,0.09,0.22,1,1,1,1
5,1.51768,12.65,3.56,1.30,73.08,0.61,8.69,0.00,0.14,1,1,1,1
6,1.51769,12.45,2.71,1.29,73.70,0.56,9.06,0.00,0.24,1,1,1,1
7,1.51892,13.46,3.83,1.26,72.55,0.57,8.21,0.00,0.14,1,1,1,1
8,1.51727,14.70,0.00,2.34,73.28,0.00,8.95,0.66,0.00,2,2,2,1
9,1.51545,14.14,0.00,2.68,73.39,0.08,9.07,0.61,0.05,2,2,2,1


In [23]:
#Getting the classes of 20th, 60th, 100th, 140th, and 180th examples
data_output.iloc[19:180:40, [11, 10]]

,Predicted_Class_CV_GNB,Predicted_Class_GNB
indextoUse,,
20,1,1
60,1,1
100,1,1
140,2,2
180,2,2


In [24]:
#Saving the results table as a csv file
#data_output.to_csv('predictionsHW1.csv')